In [2]:
import sympy as sp
import numpy as np
from tensorflow.keras.models import load_model
import joblib

In [4]:
# Cargar el modelo y los escaladores
model = load_model('../best_model_2.4367305027108534e-05.h5')
scaler_X = joblib.load('../minmax_scaler_X.pkl')
scaler_y = joblib.load('../minmax_scaler_y.pkl')

c:\Users\PcR\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Se generará un txt con la función de activación porque es muy amplia

In [3]:
# Extraer pesos y sesgos
W1, b1 = model.layers[0].get_weights()
W2, b2 = model.layers[1].get_weights()
W3, b3 = model.layers[2].get_weights()

# Definir variables simbólicas para la entrada
y1, G_, wt_mea, z, T_gas, L_ = sp.symbols('y1 G_ wt_mea z T_gas L_')
x = sp.Matrix([y1, G_, wt_mea, z, T_gas, L_])

# Funciones de activación
def relu(x):
    return sp.Max(x, 0)

def sigmoid(x):
    return 1 / (1 + sp.exp(-x))

# Construir la función de la red neuronal capa por capa
def build_neural_network_function():
    # Normalización
    x_norm = sp.Matrix([(xi - min_i) / (max_i - min_i) 
                        for xi, min_i, max_i in zip(x, scaler_X.min_, scaler_X.data_max_)])
    
    # Primera capa
    z1 = sp.Matrix(W1).T * x_norm + sp.Matrix(b1)
    a1 = z1.applyfunc(relu)
    
    # Segunda capa
    z2 = sp.Matrix(W2).T * a1 + sp.Matrix(b2)
    a2 = z2.applyfunc(sigmoid)
    
    # Capa de salida
    z3 = sp.Matrix(W3).T * a2 + sp.Matrix(b3)
    
    # Desnormalización
    output = z3[0] * (scaler_y.data_max_[0] - scaler_y.min_[0]) + scaler_y.min_[0]
    
    return output

# Construir la función
print("Generando la función matemática completa...")
nn_function = build_neural_network_function()

# Guardar la función en un archivo de texto
output_file = 'neural_network_function.txt'
print(f"Guardando la función en {output_file}...")

with open(output_file, 'w') as file:
    file.write("Función matemática completa de la red neuronal:\n\n")
    file.write(str(nn_function))

print(f"La función completa ha sido guardada en {output_file}")

# Información adicional sobre la estructura de la red
print("\nEstructura de la red:")
print(f"Entrada: {len(W1)} neuronas")
print(f"Capa oculta 1: {len(W2)} neuronas con activación ReLU")
print(f"Capa oculta 2: {len(W3)} neuronas con activación Sigmoid")
print("Capa de salida: 1 neurona con activación lineal")

c:\Users\PcR\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Generando la función matemática completa...
Guardando la función en neural_network_function.txt...
La función completa ha sido guardada en neural_network_function.txt

Estructura de la red:
Entrada: 6 neuronas
Capa oculta 1: 872 neuronas con activación ReLU
Capa oculta 2: 864 neuronas con activación Sigmoid
Capa de salida: 1 neurona con activación lineal


In [5]:

# Extraer pesos y sesgos
W1, b1 = model.layers[0].get_weights()
W2, b2 = model.layers[1].get_weights()
W3, b3 = model.layers[2].get_weights()

# Definir variables simbólicas para la entrada
y1, G_, wt_mea, z, T_gas, L_ = sp.symbols('y1 G_ wt_mea z T_gas L_')
x = sp.Matrix([y1, G_, wt_mea, z, T_gas, L_])

# Funciones de activación
def relu(x):
    return sp.Max(x, 0)

def sigmoid(x):
    return 1 / (1 + sp.exp(-x))

# Función para redondear números a una precisión específica
def round_expr(expr, num_decimals=6):
    return expr.xreplace({n : round(n, num_decimals) for n in expr.atoms(sp.Number)})

# Construir la función de la red neuronal capa por capa con simplificación
def build_simplified_neural_network_function():
    # Normalización
    x_norm = sp.Matrix([(xi - min_i) / (max_i - min_i) 
                        for xi, min_i, max_i in zip(x, scaler_X.min_, scaler_X.data_max_)])
    
    # Primera capa
    z1 = round_expr(sp.Matrix(W1).T * x_norm + sp.Matrix(b1))
    a1 = z1.applyfunc(relu)
    
    # Segunda capa
    z2 = round_expr(sp.Matrix(W2).T * a1 + sp.Matrix(b2))
    a2 = z2.applyfunc(sigmoid)
    
    # Capa de salida
    z3 = round_expr(sp.Matrix(W3).T * a2 + sp.Matrix(b3))
    
    # Desnormalización
    output = round_expr(z3[0] * (scaler_y.data_max_[0] - scaler_y.min_[0]) + scaler_y.min_[0])
    
    return output

# Construir la función simplificada
print("Generando la función matemática simplificada...")
nn_function = build_simplified_neural_network_function()

# Simplificar la expresión
print("Simplificando la expresión...")
nn_function_simplified = sp.simplify(nn_function)

# Guardar la función simplificada en un archivo de texto
output_file = 'neural_network_function_simplified.txt'
print(f"Guardando la función simplificada en {output_file}...")

with open(output_file, 'w') as file:
    file.write("Función matemática simplificada de la red neuronal:\n\n")
    file.write(str(nn_function_simplified))

print(f"La función simplificada ha sido guardada en {output_file}")

# Información adicional sobre la estructura de la red
print("\nEstructura de la red:")
print(f"Entrada: {len(W1)} neuronas")
print(f"Capa oculta 1: {len(W2)} neuronas con activación ReLU")
print(f"Capa oculta 2: {len(W3)} neuronas con activación Sigmoid")
print("Capa de salida: 1 neurona con activación lineal")

Generando la función matemática simplificada...


KeyboardInterrupt: 